In [307]:
#Importing libraries.

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import shutup
import requests
from bs4 import BeautifulSoup
shutup.please()
import lxml.html
import re

In [252]:
link = 'https://www.eltiempo.es/en-provincia-almeria/abejuela.html'
driver = webdriver.Chrome()
driver.get(link)

In [256]:
driver.find_elements(By.XPATH, '//*[@id="headerCity"]/div[2]/section/div[1]/section[1]/span')[0].text

'22°'

In [57]:
links_auto = []

links = driver.find_elements(By.XPATH,  "//a[contains(@href, 'provincia')]")

for i in links:
    links_auto.append(i.get_attribute('href'))

In [58]:
links_manual = ['https://www.eltiempo.es/navarra',
                'https://www.eltiempo.es/melilla',
                'https://www.eltiempo.es/ceuta',
                'https://www.eltiempo.es/alava',
                'https://www.eltiempo.es/guipuzcoa',
                'https://www.eltiempo.es/vizcaya',
                'https://www.eltiempo.es/a-coruna',
                'https://www.eltiempo.es/ourense',
                'https://www.eltiempo.es/baleares',
                'https://www.eltiempo.es/la-rioja',
                'https://www.eltiempo.es/asturias',
                'https://www.eltiempo.es/murcia']

In [59]:
provinces = links_auto + links_manual

In [60]:
len(provinces)

52

In [141]:
driver.find_element(By.XPATH, '//*[@id="main"]/div[4]/div/section[6]/article/div/div[1]/ul/li[1]/a').get_attribute('href')

'https://www.eltiempo.es/aballe.html'

In [162]:
links = []

for i in tqdm(provinces):
    driver.get(i)
    time.sleep(5)
    
    xpath_a = '//*[@id="main"]/div[4]/div/section[6]/article/div/div['
    xpath_b = ']/ul/li['
    xpath_c = ']/a'
    
    for n in range(25):
        xpath_base = xpath_a + str(n) + xpath_b
        for p in range(1000):
            try:
                xpath_full = xpath_base + str(p) + xpath_c
                town_link = driver.find_element(By.XPATH, xpath_full).get_attribute('href')
                links.append(town_link)
            except:
                pass

100%|███████████████████████████████████████████████████████████████████████████████| 52/52 [4:45:32<00:00, 329.46s/it]


In [243]:
# Dropping duplicates.

links = list(set(links))

In [168]:
# Accessing the first link for testing purposes.

page = requests.get(links[0])

## Approach 1

In [169]:
links[0]

'https://www.eltiempo.es/en-provincia-almeria/abejuela.html'

In [207]:
soup = BeautifulSoup(page.content, 'lxml')

In [208]:
# Using this string to verify that the link corresponds to an actual town.

'El tiempo en' in soup.find_all('h1')[0].get_text()

True

In [264]:
# Extracting the province and cleaning the resulting string.

province = soup.find_all(class_="province")[0].get_text().split("\n")[2]
province

'                        Almería                             provincia'

In [267]:
# Town name.

soup.find_all('h1')[0].get_text().split("\n")[4]

'                    Abejuela'

##  Approach 2

Using one of the site's network calls we can obtain a dictionary with all meteo data in real time, but first we'll have to find the **pelmorex_id** of every town so that we can perform the call. Let's use regex to create a dictionary containing **town url**, **name** and **id**.

In [305]:
# Grabbing the javascript text containing the town name and pelmorex_id.

script = soup.find_all('script', {'type': 'text/javascript'})[2].text
script

'\n        var current_location_info = {};\n                                            // prepare all the data for the PSS alert system\n        current_location_info[\'foreca_id\'] = "102522495";\n        current_location_info[\'urlized\'] = "en-provincia-almeria/abejuela";\n        current_location_info[\'pelmorex_id\'] = "ESXX16293";\n        current_location_info[\'name\'] = "Abejuela";\n        current_location_info[\'region_name\'] = "Almería";\n        current_location_info[\'country_name\'] = "";\n        current_location_info[\'pss\'] = "";\n                    current_location_info[\'url\'] = "/en-provincia-almeria/abejuela.html";\n                    current_location_info[\'weather\'] = "";\n        current_location_info[\'timestamp\'] = "";\n        current_location_info[\'city_page_pelmorex_id\'] = "ESXX16293";\n        current_location_info[\'is_gps\'] = false;\n                    '

In [312]:
# Isolating the palmorex_id.

result = re.search("""pelmorex_id\'] = "(.*)";\n        current_location""", script)
pelmorex_id = result.group(1)
pelmorex_id

'ESXX16293'

In [318]:
# Performing the same operation to extract the town name.

result = re.search("""name\'] = "(.*)";\n        current_location""", script)
town_name = result.group(1)
town_name

'Abejuela'

In [321]:
# And finally, province.

result = re.search("""region_name\'] = "(.*)";\n        current_location""", script)
province = result.group(1)
province

'Almería'

It seems like this approach *might* be viable. 

Now we need to visit every link from our list and, if it's a town, perform the following operations:

    - Parse page, grab town name
    - Grab town pelmorex_id
    - Grab town province
    - Create a dictionary with url + previous data
    - Store dictionary in a list

In [331]:
# Creating our loop.

town_dicts = [] # Holding the dictionaries containing all scraped data
fail = [] # Storing failed links
id_counter = [] # Storing IDs as to eliminate duplicate values.

for i in tqdm(links):
    page = requests.get(i) # Accessing the link.
    soup = BeautifulSoup(page.content, 'lxml') # Parsing it as usual
    # If the link points to a town we'll create and store the dictionary as follows
    try:
        if 'El tiempo en' in soup.find_all('h1')[0].get_text(): # Using the previous condition to filter the links
            try:
                script = soup.find_all('script', {'type': 'text/javascript'})[2].text # Locating the javascript text
                result = re.search("""pelmorex_id\'] = "(.*)";\n        current_location""", script)
                pelmorex_id = result.group(1) # Saving pelmorex_id
                result = re.search("""name\'] = "(.*)";\n        current_location""", script)
                town_name = result.group(1) # Town name
                result = re.search("""region_name\'] = "(.*)";\n        current_location""", script)
                province = result.group(1) # Province
                
                if pelmorex_id not in id_counter: # Checking if it's a duplicate.
                    # Now we can create and save the dictionary
                    town = {'url': i, 'pelmorex_id': pelmorex_id, 'name': town_name, 'province': province}
                    town_dicts.append(town) # Appending it to the list
                    id_counter.append(pelmorex_id)
                else:
                    pass
            except:
                fail.append(i) # If the loop fails we'll see at which url ir crashed 

        else:
            pass
    except:
        print(i)
        pass

 97%|███████████████████████████████████████████████████████████████████████▍  | 23996/24859 [4:12:30<07:46,  1.85it/s]

https://www.eltiempo.es/martes.html


100%|██████████████████████████████████████████████████████████████████████████| 24859/24859 [4:21:33<00:00,  1.58it/s]


In [332]:
len(town_dicts)

24858

In [333]:
fail

[]

In [334]:
town_dicts

[{'url': 'https://www.eltiempo.es/mallecina.html',
  'pelmorex_id': 'ESO1194',
  'name': 'Mallecina',
  'province': 'Asturias'},
 {'url': 'https://www.eltiempo.es/arcocha.html',
  'pelmorex_id': 'ESXX9534',
  'name': 'Arcocha',
  'province': 'Vizcaya'},
 {'url': 'https://www.eltiempo.es/torrejon.html',
  'pelmorex_id': 'ESXX16949',
  'name': 'Torrejón',
  'province': 'Teruel'},
 {'url': 'https://www.eltiempo.es/pousacarro.html',
  'pelmorex_id': 'ESXX11553',
  'name': 'Pousacarro',
  'province': 'A Coruña'},
 {'url': 'https://www.eltiempo.es/lomo-pelado.html',
  'pelmorex_id': 'ESXX12715',
  'name': 'Lomo Pelado',
  'province': 'Tenerife'},
 {'url': 'https://www.eltiempo.es/fonteferreira.html',
  'pelmorex_id': 'ESXX10487',
  'name': 'Fonteferreira',
  'province': 'Lugo'},
 {'url': 'https://www.eltiempo.es/casillas-de-flores.html',
  'pelmorex_id': 'ESXX1741',
  'name': 'Casillas de Flores',
  'province': 'Salamanca'},
 {'url': 'https://www.eltiempo.es/ceclavin.html',
  'pelmorex_id': 

In [338]:
town_index = pd.DataFrame(town_dicts)

In [339]:
town_index.to_csv('town_index.csv', index=False)